In [1]:
import pandas as pd

# Definindo nome para as colunas
r_cols = ['user_id', 'movie_id', 'rating']

# lendo o arquivo de dados csv, aplicando o nome das colunas com o nome que definimos anteriormente, já o usecols é para selecionar as colunas que queremos ler
ratings = pd.read_csv('arquivo_trello/u.data', sep='\t', names=r_cols, usecols=range(3))

# mostrando o topo do dataframe
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [2]:
import numpy as np

# agrupando pelo id do filmem e agregando as avalicaoes com a quantidade\ de vezes que o filme foi avaliado e a media das avaliacoes
movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()

/tmp/ipykernel_8238/1483366006.py:4: FutureWarning: The provided callable <function mean at 0x7fc67e5a8720> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})


rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [3]:
# criando um dataframe com o numero de avaliacoes por filme e a quantidade de vezes que foi avaliado
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
# normalizando o número de avaliacoes onde o valor mínimo será 0 e o máximo será 1
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


In [4]:
# criando um dicionário com as informações dos filmes
movieDict = {}
# abrindo o arquivo com as info dos filmes, com encoding latin-1 pq tava dando erro com caracteres especiais no utf-8
with open(r'arquivo_trello/u.item', encoding='latin-1') as f:
    temp = ''
    # para cada linha do arquivo
    for line in f:
        # campos separados por |
        fields = line.rstrip('\n').split('|')

        #pegando as informações do filme
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        # convertendo os generos para uma lista de int
        genres = list(map(int, genres))

        # adicionando as informações no dicionário
        movieDict[movieID] = (name, genres,movieNormalizedNumRatings.loc[movieID].get('size'),movieProperties.loc[movieID].rating.get('mean'))

In [5]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 np.float64(0.7735849056603774),
 np.float64(3.8783185840707963))

In [6]:
from scipy import spatial

# função que calcula a distancia entre dois filmes
def ComputeDistance(a, b):

    # generos dos filmes A e B
    genresA = a[1]
    genresB = b[1]

    # comparando a distancia dos generos através da disgtancia de cosseno, ou seja, a dessemelhança entre os generos
    genreDistance = spatial.distance.cosine(genresA, genresB)

    # popularidade dos filmes A e B
    popularityA = a[2]  
    popularityB = b[2]

    # diferença absoluta entre as popularidades (apenas a diferença, sem valor negativo)
    popularityDistance = abs(popularityA - popularityB)
    
    # retorna a distancia do genero + a distancia da popularidade
    return genreDistance + popularityDistance

# testando com dois filmes diferentes
ComputeDistance(movieDict[2], movieDict[4])

np.float64(0.8004574042309892)

In [7]:
print(movieDict[2])
print(movieDict[4])

('GoldenEye (1995)', [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], np.float64(0.22298456260720412), np.float64(3.2061068702290076))
('Get Shorty (1995)', [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], np.float64(0.3567753001715266), np.float64(3.550239234449761))


In [8]:
import operator

# Pega nos filmes "vizinhos" do filme passado como parametro
def getNeighbors(movieID, K):

    # uma lista de tuplas com as distâncias e o filme
    distances = []

    # para cada filme no dicionário
    for movie in movieDict:

        # se o filme for diferente do filme passado como parametro
        if (movie != movieID):

            # calcula a distancia entre os dois filmes
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            # adiciona na lista de distancias o filme e a distancia
            distances.append((movie, dist))

    # ordena a lista de distancias pelo segundo item da tupla (a distancia)
    distances.sort(key=operator.itemgetter(1))

    # pega os K vizinhos mais próximos
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors


# teste prático com o filme 1
K = 10
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print(movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))

avgRating /= float(K)

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463


In [9]:
avgRating

np.float64(3.3445905900235564)

In [10]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 np.float64(0.7735849056603774),
 np.float64(3.8783185840707963))

### Activity

1. Our choice of 10 for K was somewhat arbitrary - what effect do different K values have on the results?

2. Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?

In [11]:
# K é apenas a quantidade de vizinhos que serão retornados da função getNeighbors. 
# Isso impacta na quantidade que vai ser mostrado depois mas também na média final das avaliações (rating).

In [ ]:
# versao 2


from scipy import spatial

# função que calcula a distancia entre dois filmes
def ComputeDistance2(a, b):

    # generos dos filmes A e B
    genresA = a[1]
    genresB = b[1]

    # comparando a distancia dos generos através da disgtancia de cosseno, ou seja, a dessemelhança entre os generos
    genreDistance = spatial.distance.cosine(genresA, genresB)

    # popularidade dos filmes A e B
    popularityA = a[2]  
    popularityB = b[2]

    # diferença absoluta entre as popularidades (apenas a diferença, sem valor negativo)
    popularityDistance = abs(popularityA - popularityB)

    # avaliação dos filmes a e b
    ratingA = a[3]
    ratingB = b[3]

    # distantica das avaliações entre os filmes
    # a lógica é que a qualidade do filme também importa para procurar similaridade, então devemos bucas filmes que também tenha 
    # uma avaliação parecida

    # divindo por 5 para normalizar o valor entre 0 e 1
    ratingDistance = abs(ratingA - ratingB) / 5.0
    
    # colocando pesos diferentes para cada atributo
    genre_weight = 0.5
    popularity_weight = 0.25
    rating_weight = 0.25

    # retorna a distancia do genero + a distancia da popularidade
    return (genreDistance * genre_weight) + (popularityDistance * popularity_weight) + (ratingDistance * rating_weight)

# testando com dois filmes diferentes
ComputeDistance2(movieDict[2], movieDict[4])

np.float64(0.3839876359354516)